In [4]:
import pandas as pd

# List of file paths for each yearly dataset
yearly_datasets = ["/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2013-14 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2014-15 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2015-16 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2016-17 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2017-18 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2018-19 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2019-20 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2020-21 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2021-22 States Source (AwardYearSummary).xlsx",
                   "/workspaces/codespaces-jupyter/TSA_DataScience_23-24/States_Dataset/AY2022-23 States Source (AwardYearSummary).xlsx"]

# First yearly dataset is for AY 2014
year = 2014

# Iterate over each yearly dataset
for dataset_path in yearly_datasets:
    # Read the source dataset for the current year
    df_source = pd.read_excel(dataset_path)
    
    # Converting data types of data value (Recipient/Disbursement) columns
    for column in df_source.columns[2:]:
        if df_source[column].dtype == object:
            df_source[column] = pd.to_numeric(df_source[column], errors='coerce')

    # Group by the 'School Type' column and aggregate disbursement information
    df_aggregated = df_source.groupby(['State', 'School']).agg({
        'DL Subsidized: Recipients': 'sum',
        'DL Subsidized: $ of Disbursements': 'sum',
        'DL Unsubsidized (Undergraduate): Recipients': 'sum',
        'DL Unsubsidized (Undergraduate): $ of Disbursements': 'sum',
        'DL Unsubsidized (Graduate): Recipients': 'sum',
        'DL Unsubsidized (Graduate): $ of Disbursements': 'sum',
        'DL PLUS (Parent): Recipients': 'sum',
        'DL PLUS (Parent): $ of Disbursements': 'sum',
        'DL PLUS (Graduate): Recipients': 'sum',
        'DL PLUS (Graduate): $ of Disbursements': 'sum',
    }).reset_index()
    
    # Calculate the Average Disbursements per Recipient for each type of Disbursement
    df_aggregated['DL Subsidized: Average Disbursements per Recipient'] = df_aggregated.apply(lambda row: row['DL Subsidized: $ of Disbursements'] / row['DL Subsidized: Recipients'] if row['DL Subsidized: Recipients'] != 0 else 0, axis=1)
    df_aggregated['DL Unsubsidized (Undergraduate): Average Disbursements per Recipient'] = df_aggregated.apply(lambda row: row['DL Unsubsidized (Undergraduate): $ of Disbursements'] / row['DL Unsubsidized (Undergraduate): Recipients'] if row['DL Unsubsidized (Undergraduate): Recipients'] != 0 else 0, axis=1)
    df_aggregated['DL Unsubsidized (Graduate): Average Disbursements per Recipient'] = df_aggregated.apply(lambda row: row['DL Unsubsidized (Graduate): $ of Disbursements'] / row['DL Unsubsidized (Graduate): Recipients'] if row['DL Unsubsidized (Graduate): Recipients'] != 0 else 0, axis=1)
    df_aggregated['DL PLUS (Parent): Average Disbursements per Recipient'] = df_aggregated.apply(lambda row: row['DL PLUS (Parent): $ of Disbursements'] / row['DL PLUS (Parent): Recipients'] if row['DL PLUS (Parent): Recipients'] != 0 else 0, axis=1)
    df_aggregated['DL PLUS (Graduate): Average Disbursements per Recipient'] = df_aggregated.apply(lambda row: row['DL PLUS (Graduate): $ of Disbursements'] / row['DL PLUS (Graduate): Recipients'] if row['DL PLUS (Graduate): Recipients'] != 0 else 0, axis=1)
    
    # Reorder columns for the desired output format
    output_columns = ['State', 'School', 'DL Subsidized: Recipients', 'DL Subsidized: $ of Disbursements', 'DL Subsidized: Average Disbursements per Recipient',
                      'DL Unsubsidized (Undergraduate): Recipients', 'DL Unsubsidized (Undergraduate): $ of Disbursements', 'DL Unsubsidized (Undergraduate): Average Disbursements per Recipient',
                      'DL Unsubsidized (Graduate): Recipients', 'DL Unsubsidized (Graduate): $ of Disbursements', 'DL Unsubsidized (Graduate): Average Disbursements per Recipient',
                      'DL PLUS (Parent): Recipients', 'DL PLUS (Parent): $ of Disbursements', 'DL PLUS (Parent): Average Disbursements per Recipient',
                      'DL PLUS (Graduate): Recipients', 'DL PLUS (Graduate): $ of Disbursements', 'DL PLUS (Graduate): Average Disbursements per Recipient']
    df_aggregated = df_aggregated[output_columns]
    
    # Add a total row for each state at the end of each state's chunk of rows
    df_total = df_aggregated.groupby('State').agg({
        'DL Subsidized: Recipients': 'sum',
        'DL Subsidized: $ of Disbursements': 'sum',
        'DL Unsubsidized (Undergraduate): Recipients': 'sum',
        'DL Unsubsidized (Undergraduate): $ of Disbursements': 'sum',
        'DL Unsubsidized (Graduate): Recipients': 'sum',
        'DL Unsubsidized (Graduate): $ of Disbursements': 'sum',
        'DL PLUS (Parent): Recipients': 'sum',
        'DL PLUS (Parent): $ of Disbursements': 'sum',
        'DL PLUS (Graduate): Recipients': 'sum',
        'DL PLUS (Graduate): $ of Disbursements': 'sum',
    }).reset_index()

    # Calculate the Average Disbursements per Recipient for each type of Disbursement in the total row
    df_total['DL Subsidized: Average Disbursements per Recipient'] = df_total['DL Subsidized: $ of Disbursements'] / df_total['DL Subsidized: Recipients']
    df_total['DL Unsubsidized (Undergraduate): Average Disbursements per Recipient'] = df_total['DL Unsubsidized (Undergraduate): $ of Disbursements'] / df_total['DL Unsubsidized (Undergraduate): Recipients']
    df_total['DL Unsubsidized (Graduate): Average Disbursements per Recipient'] = df_total['DL Unsubsidized (Graduate): $ of Disbursements'] / df_total['DL Unsubsidized (Graduate): Recipients']
    df_total['DL PLUS (Parent): Average Disbursements per Recipient'] = df_total['DL PLUS (Parent): $ of Disbursements'] / df_total['DL PLUS (Parent): Recipients']
    df_total['DL PLUS (Graduate): Average Disbursements per Recipient'] = df_total['DL PLUS (Graduate): $ of Disbursements'] / df_total['DL PLUS (Graduate): Recipients']

    # Add 'School' column to the total row
    df_total['School'] = 'TOTAL'

    # Reorder columns for the desired output format
    df_total = df_total[output_columns]
    
    # Concatenate the total row to the original aggregated DataFrame
    df_aggregated = pd.concat([df_aggregated, df_total], ignore_index=True)
    
    # Replace NaN with 0
    df_aggregated.fillna(0, inplace=True)
    
    # Save the individual DataFrame for each year to an Excel file
    df_aggregated.to_excel(f'output_combined_dataset_{year}.xlsx', index=False)

    # Increase the year by one for the next iteration
    year += 1
